In [ ]:
import pandas as pd

from pathlib import Path

import pandas as pd

from kinfraglib import utils

In [83]:
HERE = Path(_dh[-1])
PATH_DATA = HERE / '../../data'

In [84]:
fragment_library_custom = utils.read_fragment_library(PATH_DATA / "fragment_library_custom_filtered")
fragment_library_custom.keys()

dict_keys(['AP', 'FP', 'SE', 'GA', 'B1'])

In [85]:
def standardized_inchi(mol):
    return Chem.inchi.MolToInchiKey(utils.standardize_mol(mol))
    

In [86]:
standardized_mol_inchis = {sp: list(sp_lib['ROMol'].apply(standardized_inchi)) for sp, sp_lib in fragment_library_custom.items()}

In [87]:
standardized_mol_inchis['AP']

['HCTKTFWOSSBSIL-UHFFFAOYSA-N',
 'VSPXQZSDPSOPRO-UHFFFAOYSA-N',
 'AXWBJJJQRJBTMI-UHFFFAOYSA-N',
 'LDIJKUBTLZTFRG-UHFFFAOYSA-N',
 'MXVAGCQKBDMKPG-UHFFFAOYSA-N',
 'LRBDMVYIUGZSKX-UHFFFAOYSA-N',
 'FOQWCSIQBWFFOM-UHFFFAOYSA-N',
 'YNNMZEFRJKNDNH-UHFFFAOYSA-N',
 'JJTNLWSCFYERCK-UHFFFAOYSA-N',
 'JWVCLYRUEFBMGU-UHFFFAOYSA-N',
 'FWOPJXVQGMZKEP-UHFFFAOYSA-N',
 'MVXVYAKCVDQRLW-UHFFFAOYSA-N',
 'BMTSZVZQNMNPCT-UHFFFAOYSA-N',
 'LJXQPZWIHJMPQQ-UHFFFAOYSA-N',
 'KXDAEFPNCMNJSK-UHFFFAOYSA-N',
 'DCLWNTIANRACSB-UHFFFAOYSA-N',
 'PXDDCBOOHGDLBL-UHFFFAOYSA-N',
 'QMNUDYFKZYBWQX-UHFFFAOYSA-N',
 'DRYRBWIFRVMRPV-UHFFFAOYSA-N',
 'XPHWCAKVRKUXLK-UHFFFAOYSA-N',
 'NLCZMMKTARJHDH-UHFFFAOYSA-N',
 'YDTDKKULPWTHRV-UHFFFAOYSA-N',
 'HSKNBNNWHYJGMA-UHFFFAOYSA-N',
 'MBVAHHOKMIRXLP-UHFFFAOYSA-N',
 'RAIPHJJURHTUIC-UHFFFAOYSA-N',
 'GHPODDMCSOYWNE-UHFFFAOYSA-N',
 'FTNJQNQLEGKTGD-UHFFFAOYSA-N',
 'ISWSIDIOOBJBQZ-UHFFFAOYSA-N',
 'BAXOFTOLAUCFNW-UHFFFAOYSA-N',
 'BAXOFTOLAUCFNW-UHFFFAOYSA-N',
 'MFZQJIKENSPRSJ-UHFFFAOYSA-N',
 'ZYXBIO

In [88]:
# filter kinfraglib on stand. inchis
fragment_library = utils.read_fragment_library(PATH_DATA / "fragment_library")
fragment_library.keys()

dict_keys(['AP', 'FP', 'SE', 'GA', 'B1', 'B2', 'X'])

In [89]:
for sp, sp_lib in fragment_library.items():
    sp_lib['standard_inchi'] = sp_lib['ROMol'].apply(standardized_inchi)

In [90]:
filtered_fragment_lib = {sp: fragment_library[sp][fragment_library[sp]['standard_inchi'].isin(inchis)] 
                         for sp, inchis in standardized_mol_inchis.items()}
filtered_fragment_lib.keys()

dict_keys(['AP', 'FP', 'SE', 'GA', 'B1'])

In [91]:
# subpocket wise or not?

In [92]:
special_names = {"DCLK1": "DCAMKL1"}

In [93]:
def scale_size(n, min_n, max_n, min_size, max_size):
    r_size = max_size - min_size
    r_n = max_n - min_n
    s = (n - min_n)/r_n
    return int(r_size * s + min_size)

In [98]:
def kinase_map_from_df(sp_lib, color):
    kinase_map = (sp_lib.groupby('kinase', as_index=False).size()
                    .rename(columns={'kinase': 'xName', 'size': 'num_fragments'}))

    kinase_map['xName'] = kinase_map['xName'].apply(lambda kinase : 
        special_names.get(kinase) or kinase.replace("-", "_"))

    kinase_map['shape'] = ['circle' for _ in range(len(kinase_map))]
    kinase_map['fill'] = [color for _ in range(len(kinase_map))]
    kinase_map['stroke'] = [color for _ in range(len(kinase_map))]
    kinase_map['strokeWidth'] = [color for _ in range(len(kinase_map))]
    kinase_map['fill'] = [color for _ in range(len(kinase_map))]
    
    return kinase_map

In [99]:
for sp, sp_lib_custom in filtered_fragment_lib.items():
    kinase_map = pd.concat([kinase_map_from_df(fragment_library[sp], 'blue'), kinase_map_from_df(sp_lib_custom, 'green')])
    min_n, max_n = kinase_map['num_fragments'].min(), kinase_map['num_fragments'].max()
    kinase_map['size'] = kinase_map['num_fragments'].apply(lambda n: scale_size(n, min_n, max_n, 15, 50))
    kinase_map.to_csv(sp + "_kinase_map.csv")